# Import Packages

In [12]:
import pandas as pd
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn import set_config
set_config(display='diagram')



# Reloading in the Data

In [2]:
df = pd.read_csv('Data/sales_predictions (3).csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


# Performing Preprocessing Steps

## Data Cleaning Before Data Spliting

In [3]:
# Drop unnecessary columns: these columns don't make impact on the prediction of the sales.

df.drop(columns = ['Item_Identifier', 'Outlet_Identifier', 'Outlet_Establishment_Year'],  axis = 1, inplace = True)

In [4]:
# Drop any duplicates
df.duplicated().sum()

0

In [6]:
# Checking missing values. And display the data information.
print(df.info())
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Item_Weight           7060 non-null   float64
 1   Item_Fat_Content      8523 non-null   object 
 2   Item_Visibility       8523 non-null   float64
 3   Item_Type             8523 non-null   object 
 4   Item_MRP              8523 non-null   float64
 5   Outlet_Size           6113 non-null   object 
 6   Outlet_Location_Type  8523 non-null   object 
 7   Outlet_Type           8523 non-null   object 
 8   Item_Outlet_Sales     8523 non-null   float64
dtypes: float64(4), object(5)
memory usage: 599.4+ KB
None


Item_Weight             1463
Item_Fat_Content           0
Item_Visibility            0
Item_Type                  0
Item_MRP                   0
Outlet_Size             2410
Outlet_Location_Type       0
Outlet_Type                0
Item_Outlet_Sales          0
dtype: int64

In [7]:
# the stats informaiton of the data.
df.describe(include='all')

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
count,7060.000000,8523,8523.000000,8523,8523.000000,6113,8523,8523,8523.000000
unique,NaN,5,NaN,16,NaN,3,3,4,NaN
top,NaN,Low Fat,NaN,Fruits and Vegetables,NaN,Medium,Tier 3,Supermarket Type1,NaN
freq,NaN,5089,NaN,1232,NaN,2793,3350,5577,NaN
mean,12.857645,NaN,0.066132,NaN,140.992782,NaN,NaN,NaN,2181.288914
std,4.643456,NaN,0.051598,NaN,62.275067,NaN,NaN,NaN,1706.499616
min,4.555000,NaN,0.000000,NaN,31.290000,NaN,NaN,NaN,33.290000
25%,8.773750,NaN,0.026989,NaN,93.826500,NaN,NaN,NaN,834.247400
50%,12.600000,NaN,0.053931,NaN,143.012800,NaN,NaN,NaN,1794.331000
75%,16.850000,NaN,0.094585,NaN,185.643700,NaN,NaN,NaN,3101.296400


In [8]:
# fix any inconsistent categories of data
# filter out all the object dtype columns
dtypes = df.dtypes
cat_cols = dtypes[dtypes=="object"].index
for col in cat_cols:
  print(f'-Column={col}')
  print(df[col].value_counts(dropna=False))
  print('\n\n')

-Column=Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64



-Column=Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64



-Column=Outlet_Size
Medium    2793
NaN       2410
Small     2388
High       932
Name: Outlet_Size, dtype: int64



-Column=Outlet_Location_Type
Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64



-Column=Outlet_Type
Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3  

In [9]:
# address the inconsistence in Item_Fat_Content column.
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF': 'Low Fat',
                                                        'low fat' : 'Low Fat',
                                                        'reg' : 'Regular'})
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

## Validation Split

In [10]:
# Split X and y.
X = df.drop(columns= 'Item_Outlet_Sales')
y = df['Item_Outlet_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Create a ColumnTransformer for Preprocessing

In [13]:
# Instantiate the transformers
scaler = StandardScaler()
mean_imputer = SimpleImputer(strategy='mean') # this used for dealing with the missing values in item_weight column.
freq_imputer = SimpleImputer(strategy='most_frequent') # this is used for dealing with the missing values in outlet_size column.
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ordinal = OrdinalEncoder()

In [14]:
# Prepare separate processing pipelines for numeric and categorical data
num_pipe = make_pipeline(mean_imputer, scaler)
nom_pipe = make_pipeline(ohe)
ord_pipe = make_pipeline(freq_imputer, ordinal)

In [15]:
# Create ColumnSelectors for the the numeric and categorical data
nominal_cols = ['Item_Type', 'Outlet_Location_Type', 'Outlet_Type']
ordinal_cols = ['Item_Fat_Content', 'Outlet_Size']
num_selector = make_column_selector(dtype_include='number')

In [16]:
# Combine the Pipelines and ColumnSelectors into tuples for the ColumnTransformer
nom_tuple = (nom_pipe, nominal_cols)
ord_tuple = (ord_pipe, ordinal_cols)
num_tuple = (num_pipe, num_selector)

In [17]:
# Create the preprocessing ColumnTransformer
preprocessor = make_column_transformer(nom_tuple, ord_tuple, num_tuple, remainder='drop')
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['Item_Type', 'Outlet_Location_Type',
                                  'Outlet_Type']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder())]),
                                 ['Item_Fat_Content', 'Outlet_Size']),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000024C8F8B38B0>)])

## Check the Result

In [18]:
# fit the column transformer on the training data.
preprocessor.fit(X_train)
# transform both training and testing data.
X_train_transformed = preprocessor.transform(X_train)

X_test_transformed = preprocessor.transform(X_test)

X_train_transformed.shape

(6392, 28)

In [19]:
# convert the training data to dataframe.
X_train_df = pd.DataFrame(X_train_transformed)
X_train_df.head().round(2)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.82,-0.71,1.83
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.56,-1.29,0.60
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,-0.13,1.81,0.24
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2.0,-1.17,-1.00,-0.95
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.53,-0.97,-0.34


In [21]:
X_train_df.isna().sum().sum()

0